In [1]:
from __future__ import print_function
import tensorflow as tf

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
print (type(mnist.train.images))
print (mnist.train.images.shape)

<type 'numpy.ndarray'>
(55000, 784)


In [4]:
print (mnist.train.images[0])
print (type(mnist.train.images[0]))
print (mnist.train.images[0].shape)    

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [5]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input], name='input_data')
y = tf.placeholder(tf.float32, [None, n_classes], name='output_class')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')  # dropout (keep probability)

In [7]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [8]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    with tf.name_scope('array_reshape') as array_reshape:
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    with tf.name_scope('conv_layer1') as conv_layer1:
        conv1 = conv2d(x, weights['wc1'], biases['bc1'])
        # Max Pooling (down-sampling)
        conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    with tf.name_scope('conv_layer2') as conv_layer2:
        conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
        # Max Pooling (down-sampling)
        conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    with tf.name_scope('fc_layer') as fc_layer:
        fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
        fc1 = tf.nn.relu(fc1)
        # Apply Dropout
        fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    with tf.name_scope('output_layer') as output_layer:
        out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out


In [9]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32]), name='wc1'),
    
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name='wc2'),
    
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024]), name='wd1'),
    
    # 1024 inputs, 48 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes]), name='wo1')
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32]), name='bc1'),
    'bc2': tf.Variable(tf.random_normal([64]), name='bc2'),
    'bd1': tf.Variable(tf.random_normal([1024]), name='bd1'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='bo1')
}

In [10]:
param_list = [weights["wc1"], biases["bc1"], 
              weights["wc2"], biases["bc2"], 
              weights["wd1"], biases["bd1"], 
              weights["out"], biases["out"]]
saver = tf.train.Saver(param_list)

In [11]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    tf.summary.scalar('cost/', cost)
with tf.name_scope('optimization'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
with tf.name_scope('evaluation'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    rmse = tf.metrics.root_mean_squared_error(labels=y, predictions=pred)


In [12]:
# Launch the graph
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # Initializing the variables
    init = tf.global_variables_initializer()
    sess.run(init)
    
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./summary', sess.graph)  
    
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        summary, _ = sess.run([merged, optimizer], 
                              feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        train_writer.add_summary(summary, step * batch_size)
        saver.save(sess, './tensorflow.ckpt')
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], 
                                      feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print("Iter " + str(step * batch_size))
            print ("Minibatch Loss= " + "{:.6f}".format(loss))
            print ("Training Accuracy= " + "{:.5f}".format(acc))
#             print ("RMSE= " + "{:.5f}".format(acc))
            print
        step += 1

#     for i in xrange(0,20000):
#         loss, acc, err = sess.run([cost, accuracy, rmse],
#                                   feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
#         if i/100 == 0:
#             print ("iter= " + i)
#             print ("loss= " + "{:.6f}".format(loss))
#             print ("acc= " + "{:.6f}".format(lacc))
#             print ("rmse= " + "{:.6f}".format(err))

    coord.request_stop()
    coord.join(thread)
            
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                        y: mnist.test.labels[:256],
                                        keep_prob: 1.}))

Iter 1280
Minibatch Loss= 19064.607422
Training Accuracy= 0.32812
Iter 2560
Minibatch Loss= 11226.175781
Training Accuracy= 0.55469
Iter 3840
Minibatch Loss= 8094.711426
Training Accuracy= 0.60156
Iter 5120
Minibatch Loss= 6091.456543
Training Accuracy= 0.69531
Iter 6400
Minibatch Loss= 3598.299316
Training Accuracy= 0.77344
Iter 7680
Minibatch Loss= 6737.310547
Training Accuracy= 0.70312
Iter 8960
Minibatch Loss= 2905.686523
Training Accuracy= 0.78906
Iter 10240
Minibatch Loss= 3830.239014
Training Accuracy= 0.78125
Iter 11520
Minibatch Loss= 1440.343872
Training Accuracy= 0.90625
Iter 12800
Minibatch Loss= 2730.257324
Training Accuracy= 0.83594
Iter 14080
Minibatch Loss= 1594.744751
Training Accuracy= 0.89062
Iter 15360
Minibatch Loss= 1362.139038
Training Accuracy= 0.91406
Iter 16640
Minibatch Loss= 1820.742310
Training Accuracy= 0.90625
Iter 17920
Minibatch Loss= 1374.810791
Training Accuracy= 0.88281
Iter 19200
Minibatch Loss= 1345.087280
Training Accuracy= 0.91406
Iter 20480
Mini

In [13]:
batch_x, batch_y = mnist.train.next_batch(batch_size)

In [14]:
print (batch_x)
print (type(batch_x))
print (batch_x.shape)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
<type 'numpy.ndarray'>
(128, 784)
